# Enhancing your online retail platform with generative AI

The purpose of this notebook is to show how generative AI can be integrated into an existing platform, in this case, an online retail store. We will explore several ways to improve user experience and increase efficiency. We will use the Jurassic-2 large language models series, which offers you the maximal flexibility and customization to your data. We will also see instances where task-specific models, which don't require any prompting or tweaking, can be integrated into your system in minutes.

Are you ready to get started?

## Imports and settings

In [1]:
import ai21
import pandas as pd
import requests

### API key
In order to run this notebook, you will need an API key for AI21 Studio. How can you get it?

Create a free account at [AI21 Studio](https://studio.ai21.com). You can see your API key in the *Account* tab.

In [2]:
# TODO: fill in your API key from your AI21 Studio account
YOUR_API_KEY = ""
ai21.api_key = YOUR_API_KEY
assert ai21.api_key != "", "You must provide an API key!"

## Generating product descriptions

For this section in the notebook, we will apply the model to create a product description for an eCommerce site.

### Prompting with instructions
**Why**? This is the most natural way for humans to interact.

**When?** Drafting, seeking for inspiration, or when the format and guidelines are "work in progress".

**How?** Just provide an instruction.

In [3]:
instruction = """Write an engaging product description for a clothing eCommerce site.
Product: Humor Men's Graphic T-Shirt.
Description:
"""

In [4]:
response = ai21.Completion.execute(model="j2-jumbo-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0,
                                   numResults=1)

print(response['completions'][0]['data']['text'])

This Humor Men's Graphic T-Shirt is perfect for anyone who wants to show their sense of humor. The shirt is made of high-quality materials and is comfortable to wear. It's also machine-washable, so it's easy to care for. The shirt comes in a variety of colors and sizes, so you're sure to find the perfect one for you.


#### Adjust the parameters

A useful parameter is the temperature. You can increase creativity by tweaking the temperature. With temperature 0, the model will always choose the most probable completion, so it will always be the same. Increasing the temperature will provide varying completions, where the completion may be different with every generation. We recommend starting with a value of 0.7.

In [5]:
response = ai21.Completion.execute(model="j2-jumbo-instruct",
                                   prompt=instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3) # this will make the model generate 3 optional completions

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

Our Humor Men's Graphic T-Shirt is a fun and unique way to show your sense of humor! This shirt features a bold, colorful design that is sure to turn heads. Whether you're wearing it to a party or just running errands, this shirt is sure to start a conversation.
This Humor Men's Graphic T-Shirt is a fun way to show your personality. The shirt features a unique graphic on the front, and is made of soft and comfortable material. It's perfect for everyday wear or as a special gift for a loved one.
This t-shirt is the perfect shirt for anyone who wants to show their love for humor. The shirt features a funny design that is sure to get laughs, and is made from high-quality materials that will last. It's the perfect gift for a friend or family member who loves to laugh.


#### Personalize your prompt

The model can generate clever descriptions based on an instruction, but these descriptions may not be aligned with your actual product. If you have special features, ask the model to include them:

In [6]:
detailed_instruction = """Write an engaging product description for a clothing eCommerce site. Make sure to include the following features in the description.
Product: Humor Men's Graphic T-Shirt.
Features:
- Soft cotton
- Short sleeve
- Have a print of Einstein's quote: "artificial intelligence is no match for natural stupidity”
Description:
"""

response = ai21.Completion.execute(model="j2-jumbo-instruct",
                                   prompt=detailed_instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This Humor Men's Graphic T-Shirt is the perfect way to show your sense of humor. The soft cotton and short sleeve make it comfortable to wear, while the print of Einstein's quote is eye-catching and thought-provoking. Whether you're dressing up or dressing down, this shirt is sure to start a conversation.
The Humor Men's Graphic T-Shirt is the perfect shirt for anyone who wants to add a bit of humor to their wardrobe. Made from soft cotton, this shirt is comfortable and stylish. The print of Einstein's quote is sure to get a few laughs, and it's a great way to show off your personality.
This t-shirt is made of soft cotton and has a short sleeve. The print of Einstein's quote is on the front. This t-shirt is perfect for those who like to look good and have a good sense of humor.


#### Try another prompt!
You can (and should) check several phrasing in order to get the best completion for your use case. Here is an example of what often refered to as "System message" prompt:

In [7]:
detailed_instruction = """You are a witty copywriter with great sales skills.
Product: Humor Men's Graphic T-Shirt.
Features:
- Funny print of Einstein's quote: "artificial intelligence is no match for natural stupidity”
- Soft cotton
- Short sleeve

Write an engaging product description for a clothing eCommerce site. Make sure to include the above features in the description.
Description:
"""

response = ai21.Completion.execute(model="j2-jumbo-instruct",
                                   prompt=detailed_instruction,
                                   maxTokens=100,
                                   temperature=0.7,
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")

This funny t-shirt is perfect for anyone who enjoys a good joke. The quote by Einstein is sure to get a laugh, and the soft and comfy cotton will keep you comfortable all day long.
This funny T-shirt is perfect for anyone who wants to make a statement. The quote is by Einstein, and it's a reminder that no matter how smart technology gets, some people will always do dumb things. The shirt is made of soft cotton, and it's comfortable to wear. It's also machine washable, so it's easy to care for.
Add some humor to your look with our "Artificial Intelligence Is No Match For Natural Stupidity" men's t-shirt. This witty tee is sure to get a chuckle out of anyone who sees it. Made from soft cotton, it's comfortable to wear and perfect for everyday wear. The short sleeve design makes it ideal for warm weather.


### Prompting with examples
**Why?** Examples are helpful in assisting the model to comprehend and generate responses that adhere to the intended format.

**When?** Examples are particularly useful when there are stringent format constraints, a well-defined objective, and an overall structure to be maintained.

**How?** To establish a pattern for the model to follow, present a few instances (“shots”) of input-output pairs in the prompt. This enables the model to mimic the pattern. Then, provide the input for a query example and allow the model to generate a suitable completion. This approach is commonly referred to as a "*few-shot prompt*".

#### Create a few-shot prompt

We will build a few-shot prompt comprised of the following:

1. Prefix with 3 examples. Each example contains the relevant inputs (a product name and some features to incorporate) and the output (an engaging product description). They are separated by "##".

2. The query inputs. An unseen product name and set of features for which we would like the model to output a new product description. These should follow the same format of the inputs in the prefix.

First, we collect some example data for the prompt prefix:

In [8]:
EXAMPLES_DATA = [
    {"product": "Humor Men's Graphic T-Shirt", 
     "features": ["Funny print", "Soft cotton", "Short sleeve"], 
     "description": "Funny is cool with this Graphic Print T-Shirt. Crafted in soft cotton fabric and featuring a classic short sleeve silhouette for a laid-back look and style, this men’s tee pairs with your favorite jeans or shorts and easily earns a spot on the graphic tee rotation—perfect for fun weekends or your everyday look!"},
    {"product": "Women's Boho Beach Dress", 
     "features": ["Midi dress", "Swing hem", "Slimming", "Special for summer"], 
     "description": "Women's Boho Beach Dress The style of midi dress with swing hem is slimming, It will make your tummy look flatter. This dress for women could not only make you feel cool, but also perfectly present the curve of female waist. Even in hot summer, it still keeps your elegant."},
    {"product":"Men's Soho Trousers", 
     "features": ["Designed by Retrolicious", "Stretch cotton fabric", "Side pockets", "Stripes print"],
     "description": "Designed with a side stripe and a modern slim fit, these Retrolicious Soho Trousers are the perfect pair for everyday wear. Its stretch cotton fabric makes it comfortable to wear all day long, while side pockets offer convenient storage."}
]

Then, we use the following helper functions to construct the prefix:

In [9]:
def list_features(features):
    return "\n".join(
        f"- {feat}" for feat in features
    )

def make_single_example(product, features, description):
    example = "Write an engaging product description for a clothing eCommerce site. Make sure to include the following features in the description.\n"
    example += f"Product: {product}\n"
    example += "Features:\n"
    example += list_features(features)
    example += "\nDescription:\n"
    example += description
    
    return example

SEPARATOR = "\n\n##\n\n"

FEW_SHOT_PREFIX = SEPARATOR.join(
    make_single_example(x["product"], x["features"], x["description"]) for x in EXAMPLES_DATA
)

And finally, we create a function to handle query inputs and create the full prompt:

In [10]:
def create_product_description_prompt(product, features):
    """
    Create a few-shot prompt to generate a product description with Jurassic-2 models given a name and features
    The prompt contains a preset sequence of examples followed by the query product name and features
    """
    return FEW_SHOT_PREFIX + SEPARATOR + make_single_example(product, features, '')  # keep the description blank and let the model generate

Let's see how this looks for a bag with specific features:

In [11]:
few_shot_prompt = create_product_description_prompt(
    product="Camp Director Crossbody Bag", 
    features=["Black canvas purse", "Rainbow space print", "Leather trim", "Two securely-zipped compartments"]
)

print(few_shot_prompt)

Write an engaging product description for a clothing eCommerce site. Make sure to include the following features in the description.
Product: Humor Men's Graphic T-Shirt
Features:
- Funny print
- Soft cotton
- Short sleeve
Description:
Funny is cool with this Graphic Print T-Shirt. Crafted in soft cotton fabric and featuring a classic short sleeve silhouette for a laid-back look and style, this men’s tee pairs with your favorite jeans or shorts and easily earns a spot on the graphic tee rotation—perfect for fun weekends or your everyday look!

##

Write an engaging product description for a clothing eCommerce site. Make sure to include the following features in the description.
Product: Women's Boho Beach Dress
Features:
- Midi dress
- Swing hem
- Slimming
- Special for summer
Description:
Women's Boho Beach Dress The style of midi dress with swing hem is slimming, It will make your tummy look flatter. This dress for women could not only make you feel cool, but also perfectly present t

In [12]:
response = ai21.Completion.execute(model="j2-jumbo-instruct",
                                   prompt=few_shot_prompt,
                                   maxTokens=100,
                                   temperature=0.7,
                                   stopSequences=[SEPARATOR],
                                   numResults=3)

for comp in response['completions']:
    print(comp['data']['text'].strip())
    print("=============")


This black canvas purse is perfect for camping enthusiasts. The rainbow space print is eye-catching, and the leather trim adds a touch of sophistication. The two securely-zipped compartments provide ample room for your belongings, and the crossbody strap is adjustable for a comfortable fit.
The Camp Director Crossbody Bag by [Brand Name] is a must-have for summer camp lovers! This black canvas purse features a rainbow space print and leather trim, with two securely-zipped compartments to keep your belongings safe. The adjustable strap allows you to wear it as a crossbody bag or over the shoulder.
This stylish crossbody bag is perfect for all your summer adventures. The canvas material is durable and easy to clean, while the rainbow space print is eye-catching and unique. The leather trim adds a touch of luxury, and the two securely-zipped compartments make it easy to keep your belongings organized. The Camp Director Crossbody Bag is perfect for a day at the beach, a casual day out, or 

Creating a good prompt is a process that involves some trial and error, but it doesn't require a PhD in AI. So don't give up if your first attempt fails to produce good results. Look for good examples and keep iterating!

## Repurpose with Paraphrase API

Writing product descriptions requires understanding your customers' needs and communicating the features and benefits of your product in an engaging, informative, and compelling manner. Also, as a copywriter, they need to keep a consistent tone in descriptions. On top of that, there is often a strict length restriction, which may cause multiple descriptions for the same product on different platforms. Offering your users different paraphrasing styles, such as shortening, can improve their productivity and help them provide high-quality results, all with their own voice and consistency.

In [13]:
# Take the product description we generated in the previous section
product_description = response['completions'][0]['data']['text']
print(product_description)

This black canvas purse is perfect for camping enthusiasts. The rainbow space print is eye-catching, and the leather trim adds a touch of sophistication. The two securely-zipped compartments provide ample room for your belongings, and the crossbody strap is adjustable for a comfortable fit.


In [14]:
# Let's make it shorter
new_desc = ai21.Paraphrase.execute(text=product_description,
                                   style="short")

In [15]:
# Paraphrase API will return up to 10 suggestion. Let's take a look at several of them:
for suggestion in new_desc['suggestions'][:4]:
    print(suggestion['text'] + "\n=============")

The rainbow space print and leather trim make this black canvas purse a great choice for campers. The crossbody strap is adjustable for a comfortable fit, and the two securely-zipped compartments provide ample space for your belongings.
Black canvas purses are perfect for camping enthusiasts. Two securely-zipped compartments provide ample room for your belongings, and the crossbody strap is adjustable for a comfortable fit.
Camping enthusiasts will love this black canvas purse. With its rainbow space print and leather trim, the crossbody strap is adjustable for a comfortable fit and two securely-zipped compartments provide ample storage.
The rainbow space print and leather trim make this black canvas purse perfect for camping enthusiasts. Two securely-zipped compartments provide plenty of room for your belongings. The crossbody strap is adjustable to fit comfortably.


## Contextual-Answers on the product page

You can utilize the **AI21 Studio Contextual Answers** model to access our advanced question answering technology, which is tailored to respond to queries within a specific document context. By doing so, it mitigates any potential factual inaccuracies that can arise with generic language models, ensuring that the answers provided are firmly rooted in the provided context document.

This model operates by receiving a document text, serving as the context, along with a question, and generating an answer exclusively based on the given context. Consequently, if the answer to your question is not present in the document, the model will indicate this fact rather than providing a false answer.

To gain an understanding of the model’s behavior, let’s consider a simple example of asking about the height of the Eiffel Tower. Most language models would typically provide an answer based on their training data. However, this model’s response is solely dependent on the context you supply. For illustrative purposes, we can employ the following modified paragraph sourced from [Wikipedia](https://en.wikipedia.org/wiki/Eiffel_Tower#:~:text=The%20Eiffel%20Tower%20(%2F%CB%88a%C9%AA,from%20the%20Champ%20de%20Mars) as the context:

In [16]:
# The question
question = "What is the height of the Eiffel tower?"

# Actual paragraph
context = "The tower is 330 metres (1,083 ft) tall,[6] about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

# The paragraph with manual changes of the height
false_context = "The tower is 3 metres (10 ft) tall,[6] about the same height as an 81-storey building, and the tallest structure in Paris. Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

# The paragraph with the height omitted
partial_context = "Its base is square, measuring 125 metres (410 ft) on each side. During its construction, the Eiffel Tower surpassed the Washington Monument to become the tallest human-made structure in the world, a title it held for 41 years until the Chrysler Building in New York City was finished in 1930. It was the first structure in the world to surpass both the 200-metre and 300-metre mark in height. Due to the addition of a broadcasting aerial at the top of the tower in 1957, it is now taller than the Chrysler Building by 5.2 metres (17 ft). Excluding transmitters, the Eiffel Tower is the second tallest free-standing structure in France after the Millau Viaduct."

In [17]:
# Wrapper for http request - this API is in open Beta
def ai21_contextual_answer(context, question):
    url = "https://api.ai21.com/studio/v1/experimental/answer"

    payload = {
        "context": context,
        "question": question
    }
    headers = {
        "accept": "application/json",
        "content-type": "application/json",
        "Authorization": f"Bearer {YOUR_API_KEY}"
    }

    return requests.post(url, json=payload, headers=headers).json()

In [18]:
# True context
true_response = ai21_contextual_answer(context=context, question=question)

print(true_response['answer'])

The Eiffel Tower is 330 metres (1,083 ft) tall, about the same height as an 81-storey building, and the tallest structure in Paris.


In [19]:
# False context
false_response = ai21_contextual_answer(context=false_context, question=question)

print(false_response['answer'])

The Eiffel Tower is 3 metres (10 ft) tall.


In [20]:
# Irrelevant context
irrelevant_response = ai21_contextual_answer(context=partial_context, question=question)

print(irrelevant_response['answer'])

Answer not in document


### The context - product page

In order to provide your users with the best possible experience, you must provide excellent customer service. Most questions users ask can be answered directly by the site's content, but it may not be obvious where to look. With our contextual answers API, all you have to do is provide it with the context the content on the website, and it will respond based solely on that!

*Note: This API is only part of the process for websites with a lot of content - you will need to retrieve the relevant context first. Here, we're assuming you did.*

For this example, we are taking the description from the following [Nike](https://www.nike.com/il/t/sportswear-tech-fleece-joggers-h2Bmxs/CU4495-010) product, as well as terms and conditions in the Nike website.

In [22]:
NIKE_PANTS_DESC = """Nike Sportswear Tech Fleece
₪399.90

ADDED WARMTH FOR YOUR FAVOURITE JOGGERS.


Ready for cooler weather, the Nike Sportswear Tech Fleece Joggers feature an updated fit perfect for everyday wear. Roomy through the thigh, this tapered design narrows at the knee to give you a comfortable feel without losing the clean, tailored look you love. Tall ribbed cuffs complete the jogger look while a zipped pocket on the right leg provides secure small-item storage and elevates the look.


Tech Fleece Fabric

Smooth on both sides, Tech Fleece offers premium warmth and an elevated look without adding weight or bulk.


Tech Fleece Pocket

Bonded tape around the zipped pocket and at-seam detailing are signature Tech Fleece details. Within the zipped pocket is an extra interior pocket to help keep your keys, cards and phone in place and easy to grab.


Tapered Fit

Room in the thighs helps you move without restriction. Tapering at the knee provides a tailored look to help your sneakers shine.


Adjustable Fit

Elastic waistband with drawcord provides a snug fit.


Product Details

Slim fit for a tailored feel
Ribbed cuffs and a tapered leg design
Body: 66% cotton/34% polyester. Pocket bag, palm side: 69% cotton/31% polyester. Pocket bag, knuckle side: 100% cotton.
Machine wash
Imported
Colour Shown: Saturn Gold/Black
Style: CU4495-700
Country/Region of Origin: Cambodia"""

NIKE_TOF = """During checkout, we'll provide an estimated delivery date. We'll also include that date in your order confirmation email.

STANDARD

Always free for signed-in Nike Members
Free for guest orders of ₪ 800 or more
₪ 40 for guest orders less than ₪ 800
Arrives in 4–10 business days
FAQs
Does my order need to go through customs?
Since your order is dispatched internationally, it has to be cleared through customs. Don't worry—just confirm your information at checkout and we'll handle clearing your order. The delivery date provided at checkout accounts for clearance times.

What about duties and taxes?
We eliminate the guesswork for you. All applicable taxes and duties are included in your order total at checkout.

ADDITIONAL INFORMATION
Orders are processed Sunday–Thursday, except for national holidays. Be aware that orders may experience longer processing and delivery times during holidays.
Orders can't be delivered to PO boxes, Nike stores, re-shippers or package-forwarding services.
Orders can't be delivered to addresses outside the country or region where they were placed, but you can shop for Nike products on a number of sites globally.
Nike By You orders arrive within six weeks—standard delivery times do not apply.
Refunds include the total value of any returned items. Please note that delivery fees are not refundable unless you receive an incorrect item.


HOW DO I GET FREE DELIVERY ON NIKE ORDERS?
Nike Members get free standard delivery on every order. Every time.

If you're not already a Nike Member, join Nike today for free. Sign in to your Member Profile when you shop Nike and we'll automatically apply free standard delivery at checkout on every order (no promo code required).

BECOME A NIKE MEMBER
If you'd prefer to shop as a guest, standard delivery is free for orders that meet a minimum order total. Please refer to our delivery options for additional information.

HOW LONG DOES IT TAKE TO GET A NIKE BY YOU ORDER?
We custom-make Nike By You sneakers and deliver them to you in six weeks or less. Delivery time varies by style—we'll provide an estimated delivery date during checkout.

GET STARTED WITH NIKE BY YOU
FAQs
Is expedited delivery an option for Nike By You orders?
No, expedited delivery isn't an option for Nike By You shoes—but they're worth the wait.

How much does delivery cost for Nike By You orders?
Nike By You order delivery costs are the same as all Nike orders. Please see our delivery options for additional information.

How do I check my order status?
You can check your order status online at any time.

Can I return my Nike By You sneakers?
Yes, please see our returns policy for more information.

DOES MY NIKE ORDER TOTAL INCLUDE DUTIES AND TAXES?
Yes, any applicable duties or taxes are included in your order total during checkout. We calculate duties and taxes according to the laws governing the delivery address. And please note, we don't accept tax-exempt orders.

DOES MY NIKE ORDER NEED TO CLEAR CUSTOMS?
Since your Nike order is dispatched from abroad, local customs regulations require verification of your national ID, passport or delivery information to clear and deliver your order. Don't worry—we make clearing your order easy.

Just enter your national ID during checkout—that's it. We'll work with the courier and customs to process and clear your order. The estimated delivery date provided with your order confirmation accounts for customs processing times.
"""

NIKE_CONTEXT = NIKE_PANTS_DESC + "\n" + NIKE_TOF

Let's see how the model answers the following questions (note that the last question is just straight up trolling):

In [23]:
questions = ["I need pants please send them fast, when will they arrive?", # Should answer
             "What do you count as a business day?", # Should answer
             "What shirt sizes do you have in stock?", # There is no info on shirts in the context
             "Are the pants made locally?", # Should answer
             "How much wood can a woodchuck chuck?"] # There is no answer in the context

In [25]:
for question in questions:
    print("Question: " + question)

    response = ai21_contextual_answer(context=NIKE_CONTEXT, question=question)

    print("Answer: " + response['answer'])
    print("=========================")

Question: I need pants please send them fast, when will they arrive?
Answer: Standard delivery times for Nike pants are 4–10 business days.
Question: What do you count as a business day?
Answer: Orders are processed Sunday–Thursday, except for national holidays.
Question: What shirt sizes do you have in stock?
Answer: Answer not in document
Question: Are the pants made locally?
Answer: The joggers Tech Fleece Joggers are made from Tech Fleece fabric in Cambodia.
Question: How much wood can a woodchuck chuck?
Answer: Answer not in document
